In [3]:
import os
# os.environ['DEVICE_ID'] = '0'
import numpy as np
import mindspore as ms
from mindspore import nn
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="CPU") # context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")

In [4]:
x = np.arange(-5, 5, 0.3)[:32].reshape((32, 1))
y = -5 * x + 0.1 * np.random.normal(loc=0.0, scale=20.0, size=x.shape)

In [8]:
net = nn.Dense(1, 1)
loss_fn = nn.loss.MSELoss()
opt = nn.optim.SGD(net.trainable_params(), learning_rate = 0.01)
with_loss = nn.WithLossCell(net, loss_fn)
train_step = nn.TrainOneStepCell(with_loss, opt).set_train()

In [17]:
for epoch in range(20):
    loss = train_step(ms.Tensor(x, ms.float32), ms.Tensor(y, ms.float32))
    print('epoch: {0}, loss is {1}'.format(epoch, loss))

RuntimeError: mindspore/ccsrc/runtime/device/cpu/kernel_select_cpu.cc:299 SetKernelInfo] Operator[SGD] is not support. Trace: 
In file /home/ma-user/anaconda3/envs/MindSpore-1.2.0-cpu/lib/python3.7/site-packages/mindspore/nn/optim/sgd.py(30)/    success = F.depend(success, opt(weight, gradient, learning_rate, accum, momentum, stat))/
In file /home/ma-user/anaconda3/envs/MindSpore-1.2.0-cpu/lib/python3.7/site-packages/mindspore/nn/optim/sgd.py(185)/            success = self.hyper_map(F.partial(_sgd_opt, self.opt, self.momentum, lr), gradients, params, accum, stat)/
In file /home/ma-user/anaconda3/envs/MindSpore-1.2.0-cpu/lib/python3.7/site-packages/mindspore/nn/wrap/cell_wrapper.py(357)/        loss = F.depend(loss, self.optimizer(grads))/


# 

In [15]:
wb = [x.asnumpy() for x in net.trainable_params()]
w, b = np.squeeze(wb[0]), np.squeeze(wb[1])
print('The true linear function is y = -5 * x + 0.1')
print('The trained linear model is y = {0} * x + {1}'.format(w, b))

for i in range(-10, 11, 5):
    print('x = {0}, predicted y = {1}'.format(i, net(ms.Tensor([[i]], ms.float32))))

The true linear function is y = -5 * x + 0.1
The trained linear model is y = -0.005656255874782801 * x + 0.0
x = -10, predicted y = [[0.05656256]]
x = -5, predicted y = [[0.02828128]]
x = 0, predicted y = [[0.]]
x = 5, predicted y = [[-0.02828128]]
x = 10, predicted y = [[-0.05656256]]


In [16]:
from matplotlib import pyplot as plt
plt.scatter(x, y, label='Samples')
plt.plot(x, w * x + b, c = 'r', label='True function')
plt.plot(x, -5 * x + 0.1, c='b', label='Trained model')
plt.legend()